<a href="https://colab.research.google.com/github/acharyashu/CROP-YIELD-PREDICTION-USEING-SVM-ALGORITHM/blob/main/%22LegalGPT_Fine_Tuned_Large_Language_Model_for_Contextual_Legal_Document_Analysis_and_Question_Answering%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install datasets transformers

# Import necessary modules
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import json # Import the json module

# Load dataset
with open("/content/legal_qa.txt") as f:
    data = json.load(f)

# Preprocess to T5 format
formatted_data = [
    {
        "input": f"question: {item['question']}  context: {item['context']}",
        "output": item["answer"]
    }
    for item in data
]

# Create Hugging Face Dataset
hf_dataset = Dataset.from_list(formatted_data)

# Load model and tokenizer
model_name = "t5-small"  # You can use 't5-base' or a custom legal T5 if available
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenization
def preprocess(example):
    inputs = tokenizer(example["input"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(example["output"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_data = hf_dataset.map(preprocess, batched=True)

# Training setup
training_args = TrainingArguments(
    output_dir="./legal_t5",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to=[] # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
)

# Train model
trainer.train()

# Define the legal QA function
def legal_qa(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the function
print(legal_qa(
    "What is Article 21 of the Indian Constitution?",
    "According to Article 21 of the Indian Constitution, every individual has the right to life and personal liberty."
))

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


every individual has the right to life and personal liberty
